In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
df = pd.read_csv("/kaggle/input/weatheraus/weatherAUS.csv")
df.sample(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
114569,2017-02-14,PearceRAAF,22.7,31.5,1.4,NaN,11.0,WSW,35.0,SW,...,56.0,1012.3,1012.0,7.0,NaN,27.0,30.8,Yes,0.2,No
61001,2012-07-19,Sale,0.4,13.9,0.2,2.0,8.2,W,43.0,NW,...,44.0,1022.4,1020.8,1.0,2.0,6.4,13.5,No,0.0,No
57176,2010-03-20,Bendigo,15.2,28.8,0.0,5.2,NaN,W,35.0,NaN,...,40.0,1020.3,1018.9,3.0,5.0,18.3,27.2,No,0.0,No
82094,2009-08-08,Brisbane,10.5,22.7,0.0,2.8,10.0,E,28.0,SW,...,47.0,1019.7,1016.7,1.0,1.0,16.4,20.5,No,0.0,No
136801,2010-11-28,Darwin,24.7,32.8,7.6,6.2,4.7,N,28.0,E,...,69.0,1008.5,1005.9,7.0,7.0,28.5,30.4,Yes,0.0,No


In [3]:
print("Size of weather dataframe:", df.shape)

Size of weather dataframe: (142193, 24)


## Data Preprocessing

In [4]:
# check for null values
print(df.count().sort_values())
# drop irrerevant columns
df = df.drop(columns = ['Sunshine','Evaporation','Cloud3pm', 'Cloud9am', 'Date', 'Location', 'RISK_MM' ])
print(df.shape)

Sunshine          74377
Evaporation       81350
Cloud3pm          85099
Cloud9am          88536
Pressure9am      128179
Pressure3pm      128212
WindDir9am       132180
WindGustDir      132863
WindGustSpeed    132923
WindDir3pm       138415
Humidity3pm      138583
Temp3pm          139467
WindSpeed3pm     139563
Humidity9am      140419
RainToday        140787
Rainfall         140787
WindSpeed9am     140845
Temp9am          141289
MinTemp          141556
MaxTemp          141871
Date             142193
Location         142193
RISK_MM          142193
RainTomorrow     142193
dtype: int64
(142193, 17)


In [5]:
# removing all null values
df = df.dropna(how = "any")
print(df.shape)

(112925, 17)


In [6]:
#dealing with outliers using z_score
from scipy import stats
#select numeric columns only with the standardized values
z = np.abs(stats.zscore(df._get_numeric_data()))
print(z)
#remove rows with extreme outliers.
df = df[(z < 3).all(axis = 1)]
print(df.shape)  

[[0.11756741 0.10822071 0.20666127 ... 1.14245477 0.08843526 0.04787026]
 [0.84180219 0.20684494 0.27640495 ... 1.04184813 0.04122846 0.31776848]
 [0.03761995 0.29277194 0.27640495 ... 0.91249673 0.55672435 0.15688743]
 ...
 [1.44940294 0.23548728 0.27640495 ... 0.58223051 1.03257127 0.34701958]
 [1.16159206 0.46462594 0.27640495 ... 0.25166583 0.78080166 0.58102838]
 [0.77784422 0.4789471  0.27640495 ... 0.2085487  0.37167606 0.56640283]]
(107868, 17)


In [7]:
df.replace({
    "RainToday": {"No": 0, "Yes": 1},
    "RainTomorrow": {"No": 0, "Yes": 1}
}, inplace=True)

/tmp/ipykernel_13/3100375108.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({


In [8]:
# unique values and convert them to int
categorical_columns = ["WindDir9am", "WindGustDir", "WindDir3pm"]
for col in categorical_columns:
    print(np.unique(df[col]))

# transform categorical columns
df = pd.get_dummies(df, columns = categorical_columns, dtype = int)
print(df.iloc[4:9])

['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
   MinTemp  MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  WindSpeed3pm  \
4     17.5     32.3       1.0           41.0           7.0          20.0   
5     14.6     29.7       0.2           56.0          19.0          24.0   
6     14.3     25.0       0.0           50.0          20.0          24.0   
7      7.7     26.7       0.0           35.0           6.0          17.0   
8      9.7     31.9       0.0           80.0           7.0          28.0   

   Humidity9am  Humidity3pm  Pressure9am  Pressure3pm  ...  WindDir3pm_NNW  \
4         82.0         33.0       1010.8       1006.0  ...               0   
5         55.0         23.0       1009.2       1005.4  ...               0   
6         49.0         19.0       1009.6       1

In [9]:
df.shape

(107868, 62)

In [10]:
# Standardize the data
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index = df.index, columns = df.columns)
print(df.iloc[1:6])

    MinTemp   MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  WindSpeed3pm  \
1  0.358289  0.518519  0.000000       0.506849      0.054054      0.476190   
2  0.505348  0.533333  0.000000       0.534247      0.459459      0.571429   
3  0.406417  0.590123  0.000000       0.232877      0.243243      0.166667   
4  0.628342  0.696296  0.035714       0.465753      0.135135      0.428571   
5  0.550802  0.632099  0.007143       0.671233      0.459459      0.523810   

   Humidity9am  Humidity3pm  Pressure9am  Pressure3pm  ...  WindDir3pm_NNW  \
1     0.370787         0.25     0.337292     0.326139  ...             0.0   
2     0.303371         0.30     0.266033     0.347722  ...             0.0   
3     0.382022         0.16     0.503563     0.446043  ...             0.0   
4     0.797753         0.33     0.342043     0.282974  ...             0.0   
5     0.494382         0.23     0.304038     0.268585  ...             0.0   

   WindDir3pm_NW  WindDir3pm_S  WindDir3pm_SE  WindDir3pm_SSE 

## Exploratory Data Analysis(EDA)

In [11]:
#feature selection
from sklearn.feature_selection import SelectKBest, chi2

X = df.loc[:, df.columns!= "RainTomorrow"]
y = df[["RainTomorrow"]]

selector = SelectKBest(chi2, k = 3)
selector.fit(X, y)

X_new = selector.transform(X)
print(X.columns[selector.get_support(indices = True)])

Index(['Rainfall', 'Humidity3pm', 'RainToday'], dtype='object')


In [12]:
df = df[['Rainfall', 'Humidity3pm', 'RainToday','RainTomorrow']]
X = df[['Humidity3pm']]
y = df[['RainTomorrow']]

## Data Modelling

### 1. Logistic Regression

In [13]:
# logistic Regression libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [14]:
#calculate the accuracy and time taken by the classifier
tO = time.time()


In [15]:
#Data spliting
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.2)

clf_logreg = LogisticRegression(random_state = 0)
# build the model using training dataset
clf_logreg.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=0)

In [16]:
#evaluate the model using test data
y_pred = clf_logreg.predict(X_test)
score = accuracy_score(y_test, y_pred)

In [17]:
#print accuracy and time taken by the classifier
print("LogisticRegression Accuracy:", score)
print("LogisticRegression Time taken:", time.time()- tO)

LogisticRegression Accuracy: 0.8401316399369612
LogisticRegression Time taken: 0.24367523193359375


### 2. Random Forest Classifier

In [18]:
# random forest libraries
from sklearn.ensemble import RandomForestClassifier

In [19]:
clf_rf= RandomForestClassifier(n_estimators = 100, max_depth = 4, random_state = 0)
# build the model using training dataset
clf_rf.fit(X_train, y_train)

/tmp/ipykernel_13/1619099864.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf_rf.fit(X_train, y_train)


RandomForestClassifier(max_depth=4, random_state=0)

In [20]:
#evaluate the model using test data
y_pred = clf_rf.predict(X_test)
score = accuracy_score(y_test, y_pred)

In [21]:
#print accuracy and time taken by the classifier
print("RandomForest Accuracy:", score)
print("RandomForest Time taken:", time.time()- tO)

RandomForest Accuracy: 0.8421247798275702
RandomForest Time taken: 3.072256565093994


### 3. Decision Tree Classifier

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
clf_dt = DecisionTreeClassifier(random_state = 0)
clf_dt.fit(X_train, y_train)

y_pred = clf_dt.predict(X_test)
score = accuracy_score(y_test, y_pred)

print("DecisionTree Accuracy:", score)
print("Time Taken for DecisionTree:", time.time() - tO)


DecisionTree Accuracy: 0.8406415129322332
Time Taken for DecisionTree: 3.1782827377319336


### 4. Support Vector Machine

In [24]:
# support vector machine libraries
from sklearn import svm

In [25]:
clf_svm = svm.SVC(kernel = "linear")
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)
score = accuracy_score(y_test, y_pred)
print("svm Accuracy:", score)
print("svm time taken:", time.time()-tO)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svm Accuracy: 0.7965606748864373
svm time taken: 171.47031831741333
